In [6]:
import os

neo4j_config = {
    "password": "password"
}

In [55]:
from copy import deepcopy
import json
import pprint
pp = pprint.PrettyPrinter(depth=6)

from py2neo import Graph
neo4j_con = Graph(**neo4j_config)

In [70]:
def get_recommendations(product_id):
    query_incoming_prods = """MATCH (p1:Product)-[r:BOUGHT_WITH]->(p2:Product) WHERE p2.product_id={product_id} 
                                AND p1.group=p2.group RETURN p1 ORDER BY p1.avg_rating DESC LIMIT 5"""
    
    query_outgoing_prods = """MATCH (p1:Product)-[r:BOUGHT_WITH]->(p2:Product) WHERE p1.product_id={product_id} 
                                AND p1.group=p2.group RETURN p2 ORDER BY p2.avg_rating DESC LIMIT 5"""
    
    cur = neo4j_con.run(query_incoming_prods, {"product_id": product_id})
    incoming_prods_nodes = []
    for node_tups in cur:
        for node_tup in node_tups.values():
            nodes_dict = {k: v for k, v in node_tup.iteritems()}
            incoming_prods_nodes.append(nodes_dict)
    
    cur = neo4j_con.run(query_outgoing_prods, {"product_id": product_id})
    outgoing_prods_nodes = []
    for node_tups in cur:
        for node_tup in node_tups.values():
            nodes_dict = {k: v for k, v in node_tup.iteritems()}
            outgoing_prods_nodes.append(nodes_dict)
    
    return incoming_prods_nodes, outgoing_prods_nodes

In [83]:
incoming_prods_nodes, outgoing_prods_nodes = get_recommendations(10)
incoming_prods_nodes.extend(outgoing_prods_nodes)
# print(incoming_prods_nodes)
incoming_prods_nodes.sort(key=lambda prod: prod['avg_rating'])
prod_ids = map(lambda prod: prod["product_id"], incoming_prods_nodes)
print("Recommended products are - " + str(prod_ids))

Recommended products are - [5, 139, 137, 14289, 11699, 111931, 88582, 137]
